# <p style="text-align:center;">Clustering des joueurs de football</p>

## Import des bibliothèques requises

In [ ]:
import sys
print(sys.executable)

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'pandas'

## Création du Data Frame

In [ ]:
df = pd.read_csv('Datas/X_Train_Data/train_away_player_statistics_df.csv')
df.describe()

## Nettoyage des données

In [ ]:

df = df.select_dtypes(include=['number']) # On selectionne seulement les features numériques

df = df.dropna(axis=1, thresh=len(df)*0.5) # On ignore les colonne comportant plus de 50 pourcent de NaN

df = df.fillna(df.mean()) # On remplace toutes les valeurs NaN par la moyenne de la colonne 



## Standardisation des données.

In [ ]:

scaler = StandardScaler() 
X_scaled = scaler.fit_transform(df) # On centre chaques features sur 0 avec écart-type 1. Elles sont donc centrées réduites.


## Méthode du coude afin d'évaluer le nombre de clusters adéquat

In [ ]:




# Méthode du coude : déterminer le nombre optimal de clusters
K_range = range(1, 15) # Nous allons choisir le nombre de clusters idéal entre 1 et 14
inertia = [KMeans(n_clusters=k, random_state=54).fit(X_scaled).inertia_ for k in K_range] # On entraine le modèle Kmeans sur notre dataset standardisée et on relève l'inertie associée
print(inertia)

# On crée un modèle KMeans avec k clusters avec Kmeans
# On l'entraîne sur les données normalisées avec .fit VOIR 1 DE L'ANNEXE
# On ajoute l'inertie à la liste avec inertia


# Affichage du graphique
plt.figure(figsize=(10,6))
plt.plot(K_range, inertia, 'rx--', markersize=8) #croix rouge
plt.xlabel('Nombre de clusters (k)')
plt.ylabel('Inertie')
plt.title('Methode du Coude')
plt.grid(True) # Pour avoir une grille
plt.tight_layout() # Sert a tous recenter
optimal_k = 4 # Afficher le choix que l'on a fait
plt.axvline(x=optimal_k, color='r', linestyle='--', label=f'Nombre optimal de clusters: {optimal_k}')
plt.show()




## Clustering avec la méthode des K-Means

In [ ]:

kmeans = KMeans(n_clusters=4, random_state=54)  # Crée une instance du modèle K-Means, avec comme paramètre 4 clusters et le générateur aléatoire 54
clusters = kmeans.fit_predict(X_scaled)
df['Cluster'] = clusters # Associe le cluster calculé par la méthode des K-Means à chaques lignes

## Utilisation de PCA (Principal Componment Analysis) afin de pouvoir projeter le clustering en 2D
Ici nous utilisons le PCA afin de réduire notre clustering à deux features principales tout en concervant autant que possible la variance des données.

In [ ]:
from sklearn.decomposition import PCA  
pca = PCA(n_components=2)
print(pca)
principal_components = pca.fit_transform(X_scaled)

In [ ]:
plt.figure(figsize=(10, 6))
sns.set_style("darkgrid")  # Utilise un style avec grille
sns.scatterplot(x=principal_components[:, 0], y=principal_components[:, 1], 
                hue=clusters, palette='Set1', s=60, alpha=0.7)

plt.title('Clustering des Joueurs', fontsize=15)
plt.xlabel('Composante Principale 1', fontsize=11)
plt.ylabel('Composante Principale 2', fontsize=11)
plt.legend(title="Cluster", title_fontsize=15)
plt.tight_layout()
plt.show()


In [ ]:
# Regrouper par cluster et prendre la moyenne
profiles = df.groupby('Cluster').mean()

# Afficher les stats moyennes pour chaque cluster
print(profiles)

# VOIR 3 DE L'ANNEXE

# ANNEXE

In [ ]:
#1.
#inertia = []
#for k in K_range:
   # modele = KMeans(n_clusters=k, random_state=54) 
    #modele.fit(X_scaled)                           
    #inertia.append(modele.inertia_)  

#2.
# ajout colonne "Cluster" 
df.describe() #voir ce que ça apporte


#3.
# cluster 0 = attaquant/aillier : drible reussi 31 , beaucoup de chance cree 
# cluster 1 = millieu defensif/ lateral : beaoup de duel gagner et moyen dans tout
# cluster 2 = defenseur : carton jaune 43 , bonne passe beaucoup , degagemnet 35
# cluster 3 = gardien : save 50 , grosse chace 0 , asist 0




## Interprétation

In [ ]:
# Regrouper par cluster et prendre la moyenne
profiles = df.groupby('Cluster').mean()

# Afficher les stats moyennes pour chaque cluster
print(profiles)

# VOIR 3 DE L'ANNEXE

In [ ]:
import plotly.express as px


# Préparer le DataFrame
df_plot = pd.DataFrame({
    'PC1': principal_components[:, 0],
    'PC2': principal_components[:, 1],
    'Cluster': clusters
})

# Associer les clusters à des noms clairs
df_plot['Profil'] = df_plot['Cluster'].map({
    0: 'Attaquants',
    1: 'Milieux',
    2: 'Défenseurs',
    3: 'Gardiens'
})

# Faire le scatter plot
fig = px.scatter(
    df_plot, x='PC1', y='PC2',
    color='Profil',
    color_discrete_sequence=px.colors.qualitative.Set1,
    title='Clustering des Joueurs',
    labels={'PC1': 'Composante Principale 1', 'PC2': 'Composante Principale 2'},
    opacity=0.8
)

fig.update_layout(legend_title_text='Profil de Joueur')
fig.show()





#### Notre objectif est de déduire quel cluster correspond à quel poste :

Attaquant

Milieu de terrain

Défenseur

Gardien de but

<span style="color: red;">Cluster 0</span>

Passes réussies : Très faible 

Dégagements (Clearances) : 2.7 assez faible

Arrêts dans la surface (Saves inside box) : 0.84 -> relativement 

Dribbles réussis : 6.7 -> correct

Tirs cadrés : 5.39

Duel total : 14.98

Jaunes : 9.59

C'est un profil plutot milieu defensif / lateral

<span style="color: blue;">Cluster 1</span>

Passes réussies : 28 -> élevé

Tacles : 26

Duel total : 37

Tirs cadrés : 36

Dribbles : 29

Jaunes : 32

Très actif offensivement.
C'est un attaquant / aillier

<span style="color: green;">Cluster 2</span>

Passes réussies : 22

Arrêts dans la surface : 50 

Tirs : très bas

Dégagements : modérés

Tacles : 1.4 -> très peu

Jaunes : 12

50 arrêts par match.
Gardien de but.

<span style="color: violet;">Cluster 3</span>

Passes réussies : 31

Tacles : 29

Duel total : 32

Tirs : 19 (moins que Cluster 1)

Très engagé défensivement.

Cela ressemble clairement à un défenseur.
